# How to Generate Race & Ethnicity Predictions using ZRP
## Geographic Identifier Level
The purpose of this notebook is to illustrate how to use ZRP submodules to generates race & ethnicity predictions. It is recommended to use `ZRP_Predict` to generate predictions, for the best coverage. There are three geographic identifier (GEOID) specific predict methods.  
- `ZRP_Predict_BlockGroup()`
- `ZRP_Predict_CensusTract()`
- `ZRP_Predict_BlockGroup()`

In this example we will illustrate how to generate predictions using `ZRP_Predict_BlockGroup`

In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi=False

In [2]:
from os.path import join, expanduser, dirname
import pandas as pd
import sys
import os
import re
import warnings

In [3]:
warnings.filterwarnings(action='once')
home = expanduser('~')

src_path = '{}/zrp'.format(home)
sys.path.append(src_path)

In [4]:
from zrp.modeling.predict import ZRP_Predict_BlockGroup
from zrp.prepare.prepare import ZRP_Prepare
from zrp.prepare.utils import load_file, load_json

## Load sample data for prediction
Load processed list of New Jersey Mayors downloaded from https://www.nj.gov/dca/home/2022mayors.csv 

In [5]:
nj_mayors = load_file("../2022-nj-mayors-sample.csv")
nj_mayors.shape

(462, 9)

In [6]:
nj_mayors

,first_name,middle_name,last_name,house_number,street_address,city,state,zip_code,ZEST_KEY
0,Gabe,NaN,Plumer,782,Frenchtown Road,Milford,NJ,08848,2
1,Ari,NaN,Bernstein,500,West Crescent Avenue,Allendale,NJ,07401,4
2,David,J.,Mclaughlin,125,Corlies Avenue,Allenhurst,NJ,07711-1049,5
3,Thomas,C.,Fritts,8,North Main Street,Allentown,NJ,08501-1607,6
4,P.,NaN,McCkelvey,49,South Greenwich Street,Alloway,NJ,08001-0425,7
...,...,...,...,...,...,...,...,...,...
457,William,NaN,Degroff,3943,Route,Chatsworth,NJ,08019,558
458,Joseph,NaN,Chukwueke,200,Cooper Avenue,Woodlynne,NJ,08107-2108,559
459,Paul,NaN,Sarlo,85,Humboldt Street,Wood-Ridge,NJ,07075-2344,560
460,Craig,NaN,Frederick,120,Village Green Drive,Woolwich Township,NJ,08085-3180,562


#### ZRP Prepare  
Predictions can only be generated from prepared data that is processed, Census GEOIDs (ie census tract), and has American Community Survey mapped to each unique record. To prepare the data we will use ZRP_Prepare

In [7]:
%%time
zest_race_predictor = ZRP_Prepare()
zest_race_predictor.fit(nj_mayors)
prepared = zest_race_predictor.transform(nj_mayors)

  0%|          | 0/462 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 15 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 170 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 420 tasks      | elapsed:    0.0s
100%|██████████| 462/462 [00:00<00:00, 15478.00it/s]

Data is loaded
   [Start] Validating input data
     Number of observations: 462
     Is key unique: True
       (Warning!!) middle_name is 68.3982683982684% missing
Directory already exists
   [Completed] Validating input data

   Formatting P1
   Formatting P2
   reduce whitespace

[Start] Preparing geo data

  The following states are included in the data: ['NJ']
   ... on state: NJ

   Data is loaded
   [Start] Processing geo data
      ...address cleaning



[Parallel(n_jobs=-1)]: Done 462 out of 462 | elapsed:    0.0s finished


      ...replicating address
         ...Base
         ...Number processing...
         House number dataframe expansion is complete! (n=462)
         ...Base
         ...Map street suffixes...
         ...Mapped & split by street suffixes...
         ...Number processing...

         Address dataframe expansion is complete! (n=900)
      ...formatting
   [Completed] Processing geo data
   [Start] Mapping geo data
      ...merge user input & lookup table
      ...mapping


100%|██████████| 1/1 [00:05<00:00,  5.03s/it]

   [Completed] Validating input geo data
Directory already exists
...Output saved
   [Completed] Mapping geo data

[Completed] Preparing geo data

[Start] Preparing ACS data
   [Start] Validating ACS input data
     Number of observations: 462
     Is key unique: True
       (Warning!!) middle_name is 68.3982683982684% missing

   [Completed] Validating ACS input data

   ...loading ACS lookup tables


   ... combining ACS & user input data
 ...Copy dataframes
 ...Block group
 ...Census tract
 ...Zip code
 ...No match
 ...Merge
 ...Merging complete
[Complete] Preparing ACS data

CPU times: user 27.6 s, sys: 2.38 s, total: 30 s
Wall time: 30 s


### Invoke the ZRP_Predict_BlockGroup on the sample data
To generate predictions, provide the path to the preferred pipeline directory in the `__init__`. Here we provide the default local path from a git clone.

In [8]:
curpath = os.getcwd()
pipe_path = join(curpath, "../../zrp/modeling/models")

In [9]:
zrp_predict =  ZRP_Predict_BlockGroup(pipe_path)

#### Load Feature List
Features are subset by GEOID
- zp: zip code
- bg: block group
- ct: census tract

In [10]:
feature_list = load_json(os.path.join(curpath, "../../zrp/modeling/feature_list_bg.json"))

We only need zip code related data so let's filter the prepared data

In [11]:
block_group_only = prepared[prepared.acs_source=='BG'].filter(feature_list)

To transform the data/generate predictions, provide the prepared data from ZRP_Prepare to the transform filtered to the `feature_list` columns

In [12]:
zrp_predict.fit()
zrp_output = zrp_predict.transform(prepared.filter(block_group_only))
zrp_output = zrp_output[~zrp_output.index.duplicated(keep='first')]

100%|██████████| 1/1 [00:00<00:00, 1120.27it/s]
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished


#### Troubleshooting
If you run into a <font color='red'>FileNotFoundError</font>, it is likely that an incorrect `pipe_path` was provided.

The pipelines are downloaded upon installation when `python -m zrp download` is ran in the terminal. Please refer to the ReadMe if you missed this step. Otherwise, the current notebook is configured to run when the pipeline is downloaded to your local zrp folder. To check if the data is downloaded to your local git cloned zrp folder run the following command.

In [13]:
!find {pipe_path}  -print | grep -i 'pipe.pkl'

/home/kam/zrp/examples/modeling/../../zrp/modeling/models/block_group/pipe.pkl
/home/kam/zrp/examples/modeling/../../zrp/modeling/models/census_tract/pipe.pkl
/home/kam/zrp/examples/modeling/../../zrp/modeling/models/zip_code/pipe.pkl


The expected output will be similar to the following 

If`!find {pipe_path}  -print | grep -i 'pipe.pkl'` returns nothing, then update your `pipe_path` to point to your virtual (conda) environment. An example is included below, where the virtal environment is named "zrp_v0.3.3":
- `pipe_path = os.path.join(home, ".conda/envs/zrp_v0.3.3/lib/python3.7/site-packages/zrp/modeling/models")`


### Inspect the output
- Preview the data
    - only one source column is expected in the output, since these predictions are GEOID specific
- View what artifacts are saved

In [14]:
zrp_output

,AAPI,AIAN,BLACK,HISPANIC,WHITE,race_proxy,source_zrp_block_group
ZEST_KEY,,,,,,,
10,0.007710,0.011133,0.019986,0.014749,0.946421,WHITE,1
100,0.010805,0.024995,0.047780,0.022430,0.893989,WHITE,1
102,0.000132,0.000109,0.000454,0.000486,0.998819,WHITE,1
103,0.078547,0.000614,0.001572,0.033324,0.885943,WHITE,1
104,0.001839,0.000730,0.008079,0.044049,0.945302,WHITE,1
...,...,...,...,...,...,...,...
93,0.038076,0.002857,0.031983,0.087664,0.839420,WHITE,1
94,0.022781,0.009774,0.002786,0.019116,0.945542,WHITE,1
95,0.033175,0.021278,0.078732,0.085143,0.781673,WHITE,1


### Check Coverage
A quick glance at the `ZRP_Predict_BlockGroup`  output we can see all records with a proper zip code that maps to American Community Survey data have a proxy

In [15]:
zrp_output.filter(regex='[A-Z]|race').isna().mean()

AAPI          0.0
AIAN          0.0
BLACK         0.0
HISPANIC      0.0
WHITE         0.0
race_proxy    0.0
dtype: float64

Checking the distribution of predicted race & ethnicity 

In [16]:
zrp_output.race_proxy.value_counts(normalize=True, dropna=False)

WHITE       0.857143
BLACK       0.077922
HISPANIC    0.032468
AAPI        0.032468
Name: race_proxy, dtype: float64

In [17]:
zrp_output.shape

(462, 7)

Please refer to the source columns to determine which geographic identifier or method was used to generate the proxy 

`ZRP_Predict_BlockGroup` automatically saves the proxies to `proxy_output.feather`